## Finding the System State as a Function of Time


## The State Vector

In the numerical solution, the state vector is stored in an array. Arrays in programming are data structures that are optimized to store numbers. The order that the components appear in the state vector isn’t important, only that we keep track of the order and don’t forget it. For the sake of choosing something, we will set the positions followed by the velocities (22):

$$y=[X_1 Y_1 Z_1 X_2 Y_2 Z_2 \dot{X_1} \dot{Y_1} \dot{Z_1} \dot{X_2} \dot{Y_2} \dot{Z_2}]$$

In the following Python code samples we use arrays to store the initial positions and velocities of both masses and then construct the state vector.

In [ ]:
import numpy as np

G = 6.67430e-20  # km**3/(kg * s**2)
m_1 = m_2 = 1.0e26  # kg

R_1_0 = np.array((0, 0, 0))  # km
R_2_0 = np.array((3000, 0, 0))  # km
dotR_1_0 = np.array((10, 20, 30))  # km/s
dotR_2_0 = np.array((0, 40, 0))  # km/s

y_0 = np.hstack((R_1_0, R_2_0, dotR_1_0, dotR_2_0))
y_0

These code samples first set the constants in the problem, $G$ and $m_1=m_2$. Then, they create the initial position and velocity arrays. Finally, the arrays are stuck together into the initial state vector, $y_0$.

### Transforming the System of Ordinary Differential Equations

The system of ordinary differential equations (ODEs) in (20) or (21) is a second order system. However, it can be transformed to a system of first-order ODEs by recognizing the acceleration as the first derivative of the velocity. Solving a first-order system is much simpler than solving a second-order system.

We know that the system of equations requires the (23)
$$\dot{y}=f(t,y)$$

where $y$ is the state vector, $t$ is time, and $f$ is a function that depends on the current time and the current state vector.

The left hand side of (23) is the time-derivative of the state vector, which we find by taking the time derivative of each element of the state vector (24):

$$\dot{y}=[\dot{X_1} \dot{Y_1} \dot{Z_1} \dot{X_2} \dot{Y_2} \dot{Z_2} \ddot{X_1} \ddot{Y_1} \ddot{Z_1} \ddot{X_2} \ddot{Y_2} \ddot{Z_2}]$$

In words, the left side of (23) is another array, where the first six elements are the velocity components (the first derivative of position) and the second six are the acceleration (the first derivative of velocity).

### Solution Algorithm

We now have enough information to start to solve the problem. The first step is to calculate the initial acceleration using (21). This can be done for one direction at a time, as shown in the following code:

In [ ]:
X_1 = y_0[0]
Y_1 = y_0[1]
Z_1 = y_0[2]
X_2 = y_0[3]
Y_2 = y_0[4]
Z_2 = y_0[5]

r = np.sqrt((X_2 - X_1) ** 2 + (Y_2 - Y_1) ** 2 + (Z_2 - Z_1) ** 2)

ddotX_1 = G * m_2 * (X_2 - X_1) / r ** 3
ddotY_1 = G * m_2 * (Y_2 - Y_1) / r ** 3
ddotZ_1 = G * m_2 * (Z_2 - Z_1) / r ** 3
ddotX_2 = -G * m_1 * (X_2 - X_1) / r ** 3
ddotY_2 = -G * m_1 * (Y_2 - Y_1) / r ** 3
ddotZ_2 = -G * m_1 * (Z_2 - Z_1) / r ** 3


This code first retrieves the position components from the state vector. Then it calculates each component of acceleration for the two masses.

This code is pretty long, and we’ve created a bunch of variables to keep track of. Fortunately, there are simpler ways to approach this calculation. The next code samples show how to take advantage of the nature of array computations:

In [ ]:
R_1 = y_0[:3]
R_2 = y_0[3:6]

r = np.sqrt(np.sum(np.square(R_2 - R_1)))
ddot = G * (R_2 - R_1) / r ** 3
ddotR_1_0 = m_2 * ddot
ddotR_2_0 = -m_1 * ddot
(R_2 - R_1)^2

In this code, you retrieve the position of each mass as an array instead of into a single variable. Then, using array functions, you compute the distance and the accelerations.

Now that you’ve calculated the acceleration, the velocity and position at the next time can be found by integration (25):

$$\dot{R_1}=\int_{0}^{t}{\ddot{R_1}}dt+\dot{R_{1,0}}$$
$$\dot{R_2}=\int_{0}^{t}{\ddot{R_2}}dt+\dot{R_{2,0}}$$
$$R_1=\int_{0}^{t}{\dot{R_1}}dt+R_{1,0}$$
$$R_2=\int_{0}^{t}{\dot{R_2}}dt+R_{2,0}$$

Assuming that the acceleration and velocity are constant over some time interval $\Delta t$, the integrals simplify to the product of the term and $\Delta t$. The choice of $\Delta t$ is up to you. Smaller values of $\Delta t$ will be more accurate but take longer to compute.

Let’s choose $\Delta t=1s$. Then, to compute the state vector at the next time step:

In [ ]:
Delta_t = 1  # s
dotR_1_1 = ddotR_1_0 * Delta_t + dotR_1_0
dotR_2_1 = ddotR_2_0 * Delta_t + dotR_2_0

R_1_1 = dotR_1_0 * Delta_t + R_1_0
R_2_1 = dotR_2_0 * Delta_t + R_2_0


However, it would be inefficient to do this by hand and there are more accurate methods available. I don’t see a reason to re-implement standard functions, so we are going to use the functions built-in to SciPy or Matlab, depending on which software you’re using.

### Numerical Solution Using Pre-Built Libraries

In SciPy, the function is called `solve_ivp`. In Matlab, the function is called `ode45`.

First, you need to start by importing the appropriate Python libraries. In Matlab, all the functions you need are built-in.

In [ ]:
#pip install orbitalpy

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

The two imports from `matplotlib` will plot the solution of the problem.

Then, we need to define the function that describes the motion of our system. This function needs to compute the left hand side of the equation above ($\dot{y}$) and return it to the solver, so that the solver can calculate the value of the state vector at time $t+\Delta t$.

The function takes the time $t$ and current value of the state vector $y$ 
as inputs. In this way, we can use the current state vector to compute the current acceleration.

Inside the function, we use the values in the state vector to fill the `ydot` vector. Then, we return `ydot` back to the solver.

In [ ]:
def absolute_motion(t, y):
    """Calculate the motion of a two-body system in an inertial reference frame.

    The state vector ``y`` should be in the order:

    1. Coordinates of $m_1$
    2. Coordinates of $m_2$
    3. Velocity components of $m_1$
    4. Velocity components of $m_2$
    """
    # Get the six coordinates for m_1 and m_2 from the state vector
    R_1 = y[:3]
    R_2 = y[3:6]

    # Fill the derivative vector with zeros
    ydot = np.zeros_like(y)

    # Set the first 6 elements of the derivative equal to the last
    # 6 elements of the state vector, which are the velocities
    ydot[:6] = y[6:]

    # Calculate the acceleration terms and fill them in to the rest
    # of the derivative array
    r = np.sqrt(np.sum(np.square(R_2 - R_1)))
    ddot = G * (R_2 - R_1) / r ** 3
    ddotR_1 = m_2 * ddot
    ddotR_2 = -m_1 * ddot

    ydot[6:9] = ddotR_1
    ydot[9:] = ddotR_2
    return ydot



With the function defined, we can call `solve_ivp()` or `ode45()`. We need to tell it the function it should solve, the beginning and end times, the initial state vector, and then some information to help control the output.

Once the solver finishes, the solution is stored in `sol.y` in Python or just `y` in Matlab. Each column of sol.y corresponds to a single timestep and each row corresponds to one of the state variables. It is more convenient to work with the transpose of this array, so we do that and define `y`. In Matlab, the solution already has each timestep in a row.

Then we extract the position and velocity of each mass as a function of time, and compute the barycenter (the center of gravity of the system).

In [ ]:
t_0 = 0  # seconds
t_f = 480  # seconds
t_points = np.linspace(t_0, t_f, 1000)

sol = solve_ivp(absolute_motion, [t_0, t_f], y_0, t_eval=t_points)

y = sol.y.T
R_1 = y[:, :3]  # km
R_2 = y[:, 3:6]  # km
V_1 = y[:, 6:9]  # km/s
V_2 = y[:, 9:]  # km/s
barycenter = (m_1 * R_1 + m_2 * R_2) / (m_1 + m_2)  # km


Finally, we construct some plots of the situation. In Fig. 9, we are plotting the absolute motion of each of the two masses as well as the barycenter. Notice that the barycenter moves in a straight line. We will discuss this further in the next section.

The two masses spiral around the barycenter. One way to imagine this system is as the Earth and the Moon viewed as though you were sitting on the Sun. The Earth and Moon would move through space, and they would appear to be orbiting around each other. If you observed them for a short enough time, their motion would appear to be in a straight line.

In [ ]:
fig = plt.figure(figsize=(5, 5), dpi=300)
ax = fig.add_subplot(111, projection="3d")
ax.plot(R_1[:, 0], R_1[:, 1], R_1[:, 2], label="m_1")
ax.plot(R_2[:, 0], R_2[:, 1], R_2[:, 2], label="m_2")
ax.plot(barycenter[:, 0], barycenter[:, 1], barycenter[:, 2], label="COG")
ax.legend()

Fig. 9 The motion of two bodies subject to mutual gravitational attraction, viewed from an external inertial frame.

Another way to view this system is by setting the barycenter to be the origin of the coordinate system, as shown in Fig. 10. Remember that since the barycenter is moving with constant velocity, it is allowed to be used as an inertial reference frame. This is kind of like sitting above the barycenter of the Earth-Moon system. You would see them orbit around the barycenter, and the orbits would be ellipses.

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(6, 6))
ax = fig.add_subplot(111, projection="3d")
R1_rel_COG = R_1 - barycenter
R2_rel_COG = R_2 - barycenter
ax.plot(R1_rel_COG[:, 0], R1_rel_COG[:, 1], R1_rel_COG[:, 2], label="m_1")
ax.plot(R2_rel_COG[:, 0], R2_rel_COG[:, 1], R2_rel_COG[:, 2], label="m_2")
ax.plot(0, 0, 0, "ro", label="COG")
ax.legend()

In [ ]:
#import plotly.io as pio

In [ ]:
import plotly.graph_objs as go
import plotly

R1_rel_COG = R_1 - barycenter
R2_rel_COG = R_2 - barycenter

trace1 = go.Scatter3d(
    x = R1_rel_COG[:, 0], 
    y = R1_rel_COG[:, 1], 
    z = R1_rel_COG[:, 2], 
    marker=dict(size=6),
    mode='markers',
    hovertext = 'R1 Vector',
    name='R1 Orbit'
) 

trace2 = go.Scatter3d(
    x = R2_rel_COG[:, 0], 
    y = R2_rel_COG[:, 1], 
    z = R2_rel_COG[:, 2],
    marker=dict(size=5),
    mode='markers',
    hovertext = 'R2 Vector',
    name='R2 Orbit'
) 

fig = go.Figure(data=[trace1, trace2])
fig.update_layout(legend_title_text='Orbits')
fig

Fig. 10 The motion of two bodies subject to mutual gravitational attraction, viewed from an inertial frame attached to the system barycenter. In this reference frame, the orbits of $m_1$ and $m_2$ appear to be ellipses with the barycenter at one of the foci.

Fig. 11 fixes the coordinate system on the first mass and plots the motion of the barycenter and the second mass relative to the position of the first mass. This is kind of like sitting on the Earth and watching the Moon go around. Notice that the barycenter of the system also orbits around the first mass in this reference frame.

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection="3d")
R2_rel_R1 = R_2 - R_1
COG_rel_R1 = barycenter - R_1
ax.plot(R2_rel_R1[:, 0], R2_rel_R1[:, 1], R2_rel_R1[:, 2], label="m_2")
ax.plot(COG_rel_R1[:, 0], COG_rel_R1[:, 1], COG_rel_R1[:, 2], label="COG")
ax.plot(0, 0, 0, "ro", label="m_1")
ax.legend()

Fig. 11 The motion of two bodies subject to mutual gravitational attraction, viewed from a non-inertial frame attached to $m_1$. In this reference frame, the orbits of the barycenter and $m_2$ appear to be ellipses with $m_1$ at one of the foci.

In [ ]:
import plotly.graph_objs as go
import plotly

R2_rel_R1 = R_2 - R_1
COG_rel_R1 = barycenter - R_1

trace1 = go.Scatter3d(
    x = R2_rel_R1[:, 0],  
    y = R2_rel_R1[:, 1], 
    z = R2_rel_R1[:, 2], 
    marker=dict(size=5),
    mode='markers',
    hovertext = 'R1 Vector',
    name='m2'
) 

trace2 = go.Scatter3d(
    x = COG_rel_R1[:, 0],   
    y = COG_rel_R1[:, 1], 
    z = COG_rel_R1[:, 2],
    marker=dict(size=5),
    mode='markers',
    hovertext = 'R2 Vector',
    name='COG'
) 

trace3 = go.Scatter3d(
    x = [0,0], 
    y = [0,0], 
    z = [0,0],
    marker=dict(size=10),
    mode='markers',
    hovertext = 'Mass 1',
    name='m1'
)

fig = go.Figure(data=[trace1, trace2, trace3])
fig.update_layout(legend_title_text='Orbits')
fig

Interestingly, the equations for this solution are symmetric. We can reverse the roles of $m_1$ and $m_2$ and have exactly the same plot as Fig. 11. This means that sitting on the Moon watching the Earth orbit is the same as sitting on the Earth watching the Moon orbit. Just like the Moon has phases when viewed from Earth, the Earth has phases when viewed from the Moon!

## Equation of Motion in a Co-moving Frame

It is difficult to define an inertial reference frame, since the vectors $R_1$ and $R_2$ are unknown in general.

In most of the problems that we work with, it is convenient to treat $m_1$ as the origin of the coordinate system. For example, in solving the motion of a satellite around Earth, we are most interested in where the satellite is relative to Earth. So a reference frame attached to the center of the Earth and moving with the Earth is quite convenient.

A reference frame attached to, and moving with,$m_1$ is shown in Fig. 12. In this reference frame, the components of $r$ are (38):

$$ r=x\hat{i}+y\hat{j}+z\hat{k}$$

Note that the position here uses lowercase $x$, $y$, and $z$ and does not involve the difference between two absolute coordinates.

Fig. 12 The co-moving coordinate system attached to $m_1$.

We can find the relative velocity and acceleration (39):

$$\dot{r}_{rel}=\dot{x}\hat{i}+\dot{y}\hat{j}+\dot{z}\hat{k}$$ and $$\ddot{r}_{rel}=\ddot{x}\hat{i}+\ddot{y}\hat{j}+\ddot{z}\hat{k}$$


The absolute acceleration is equal to the relative acceleration only in the case where $\Omega$ and 
$\dot{\Omega}$, the angular velocity and angular acceleration respectively, of the moving reference frame, are zero. Therefore, this reference frame attached to $m_1$ cannot be rotating.

Using the definition of $r$ from Eq. (38) and $\ddot{r}$ from Eq. (39), we can rewrite the equation of relative motion, Eq. (33) (40):

$$\ddot{x}=-\mu \frac{x}{r^3}$$
$$\ddot{y}=-\mu \frac{y}{r^3}$$
$$\ddot{z}=-\mu \frac{x}{r^3}$$

In the reference frame attached to $m_1$, Eq. (33) can be solved numerically in exactly the same way as Eq. (20) from Two-Body Equations of Motion in an Inertial Frame. Since the relative position, velocity, and acceleration vectors only have three components ($x$, $y$, $z$, etc.) the state vector will have 6 components instead of 12.

## Two-Body Problem in the Co-Moving Frame

Recall the equations of motion for the two-body problem in a reference frame attached to $m_1$ , Eq. (40). In this example, we will solve the two-body problem in relative coordinates. Although the problem is in relative coordinates, the solution procedure is the same as Two-Body Numerical Solution in an Inertial Frame. In Python, the first thing is to import the required libraries. This step is not necessary in Matlab.

In [ ]:
from scipy.integrate import solve_ivp
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401, needed for 3d plots

A 1000 kg satellite is in orbit around Earth. The initial position and velocity of the satellite are:

$$r_0=(8000\hat{i}+6000\hat{k}) \text{ km } $$
$$\dot{r}_{0}=7.0\hat{j} \text{ km/s } $$



We want to determine the minimum and maximum altitude, and the satellite’s speed at those two locations. The altitude of the satellite is its height above the earth’s surface. The radius of the earth is 6378.12 km.

To avoid confusion with the $y$ coordinate, we will use capital $Y$ for the state vector. Remember that the state vector includes 6 components, 3 positions and 3 velocities. These all need to be stored in one array.

In [ ]:
G = 6.67430e-20  # km**3/(kg * s**2)
m_1 = 5.97219e24  # kg
m_2 = 1000  # kg
mu = G * m_1
R_E = 6378.12  # km

r_0 = np.array((8000, 0, 6000))  # km
v_0 = np.array((0, 7, 0))  # km/s
Y_0 = np.hstack((r_0, v_0))

Then, we need to define the function that describes the motion of our system. This function needs to compute the derivative of the state vector and return it to the solver.

Inside the function, we use the values in the state vector to fill the `Ydot` vector. Then, we return `Ydot` back to the solver.

In [ ]:
def relative_motion(t, Y):
    """Calculate the motion of a two-body system relative to $m_1$.

    The state vector ``Y`` should be in the order:

    1. Coordinates of $m_2$ relative to $m_1$
    2. Velocity components of $m_2$ relative to $m_1$
    """
    # Get the three position components
    r_vec = Y[:3]

    # Create the derivative vector and copy the velocities into it
    Ydot = np.zeros_like(Y)
    Ydot[:3] = Y[3:]

    # Calculate the accelerations
    r = np.sqrt(np.sum(np.square(r_vec)))
    a_vec = -mu * r_vec / r ** 3
    Ydot[3:] = a_vec

    return Ydot

With the function defined, we can call the solver function. We need to tell it the function it should solve, the beginning and end times, the initial state vector, and then some information to help control the output.

We will guess the final time and check that increasing it does not change the answers for minimum and maximum altitude.

In Python, once the solver finishes, the solution is stored in `sol.y`. Each column of `sol.y` corresponds to a single timestep and each row corresponds to one of the state variables. It is more convenient to work with the transpose of this array, so we do that and define `Y`.

In [ ]:
t_0 = 0  # seconds
t_f = 14_709  # seconds, period of one orbit
t_points = np.linspace(t_0, t_f, 1000)
sol = solve_ivp(relative_motion, [t_0, t_f], Y_0, t_eval=t_points)

Y = sol.y.T
r = Y[:, :3]  # km
v = Y[:, 3:]  # km/s

Since we are looking for the minimum and maximum altitude, we need to calculate the distance of the satellite from the center of the earth and then subtract the radius of the earth. This will give the altitude above the surface (41):

$$h=||r||-R_E$$

where $H$ is the altitude and $R_E$ is the radius of the earth. Similarly, the speed is the magnitude of the velocity.

In [ ]:
r_mag = np.sqrt(np.sum(np.square(r), axis=1))
# altitude is the distance above the surface of the Earth
altitude = r_mag - R_E

speed = np.sqrt(np.sum(np.square(v), axis=1))


Now we can process the arrays to find the minimum and maximum altitude, and the speed at those locations. In Python, there are separate functions to find the minimum/maximum and its index; in Matlab, there is a single function to do both.

In [ ]:
min_altitude = np.min(altitude)
i_min = np.argmin(altitude)

max_altitude = np.max(altitude)
i_max = np.argmax(altitude)

speed_at_min_alt = speed[i_min]
speed_at_max_alt = speed[i_max]
time_at_min_alt = sol.t[i_min]
time_at_max_alt = sol.t[i_max]


Finally, we can print the results to the screen.

In [ ]:
print(
    f"""\
The minimum altitude during the orbit is: {min_altitude:.2F} km
The speed at the minimum altitude is: {speed_at_min_alt:.2F} km/s
The time at minimum altitude is: {time_at_min_alt:.2F} s
The maximum altitude during the orbit is: {max_altitude:.2F} km
The velocity at the maximum altitude is: {speed_at_max_alt:.4F} km/s
The time at maximum altitude is: {time_at_max_alt:.2F} s
"""
)

Interestingly, the results from Python and Matlab are slightly different. This is most likely because of different time resolution from the solvers. Now let’s plot the orbit. The central sphere is representative of the earth.

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection="3d")
ax.plot(r[:, 0], r[:, 1], r[:, 2], label="Orbit")
ax.set_xlabel("km")
ax.set_ylabel("km")
ax.set_zlabel("km")

# This adds a sphere to the plot of the radius of the Earth
p = np.linspace(0, np.pi, 200)
t = np.linspace(0, 2 * np.pi, 200)
P, T = np.meshgrid(p, t)

X = R_E * np.cos(T) * np.sin(P)
Y = R_E * np.sin(T) * np.sin(P)
Z = R_E * np.cos(P)

ax.plot_surface(X, Y, Z)
ax.plot(r[i_min, 0], r[i_min, 1], r[i_min, 2], "ro", label="Min. Altitude")
ax.plot(r[i_max, 0], r[i_max, 1], r[i_max, 2], "go", label="Max. Altitude")
ax.legend()


Fig. 13 The motion of a satellite in orbit around the earth.

In [ ]:
import plotly.graph_objs as go
import plotly

p = np.linspace(0, np.pi, 200)
t = np.linspace(0, 2 * np.pi, 200)
P, T = np.meshgrid(p, t)

X = R_E * np.cos(T) * np.sin(P)
Y = R_E * np.sin(T) * np.sin(P)
Z = R_E * np.cos(P)

fig = go.Figure(data=[go.Surface(z=Z, x=X, y=Y, showscale=False)])
    #go.Surface(x = r[:, 0], y = r[:, 1], z = r[:, 2]),
fig.add_trace(
    go.Scatter3d(x = r[:, 0], y = r[:, 1], z = r[:, 2], mode = 'markers',marker=dict(size=3), name='Orbit')) 
fig.add_trace(
    go.Scatter3d(x=(8000.0,0,0),y=(0, 0, 0),z=(6000,0,0),name='Min Altitude')) 
fig.add_trace(
    go.Scatter3d(x=(0,-12647.959804366712,0),y=(0, 23.994151853760513, 0),z=(0,-9485.969853275032,0),name='Max Altitude')) 

fig.update_layout(
    autosize=False,
    width=700,
    height=750)

fig.show()

In [ ]:
pip install PyAstronomy

In [ ]:
from __future__ import print_function, division
from PyAstronomy import pyasl
import numpy as np
import matplotlib.pylab as plt

x, y = np.linspace(-1.5, 2, 300), np.linspace(-1.6, 1.6, 300)
xx, yy = np.meshgrid(x, y)
# Coordinates in orbital plain
z = 0

# Mass ratio
q = 0.2

# Get dimensional values of Roche potential
p = pyasl.rochepot_dl(xx, yy, z, q)

# Positions (and potentials) of Lagrange points
l1, l1pot = pyasl.get_lagrange_1(q)
l2, l2pot = pyasl.get_lagrange_2(q)
l3, l3pot = pyasl.get_lagrange_3(q)
l4, l5 = pyasl.get_lagrange_4(), pyasl.get_lagrange_5()
l4pot = pyasl.rochepot_dl(l4[0], l4[1], l4[2], q)
l5pot = pyasl.rochepot_dl(l5[0], l5[1], l5[2], q)


print("Effective (dimensionless) radii of first and second mass")
print("According to the approximation of Eggleton 1983:")
r1eff = pyasl.roche_lobe_radius_eggleton(q, 1)
r2eff = pyasl.roche_lobe_radius_eggleton(q, 2)
print("    Reff1: %5.3f" % r1eff)
print("    Reff2: %5.3f" % r2eff)
print()
print("Roche volume and effective radius from Monte Carlo integration:")
mcvol1 = pyasl.roche_vol_MC(q, 1)
mcvol2 = pyasl.roche_vol_MC(q, 2)
print("    MC Roche lobe volume 1: %6.4f +/- %6.4f" % (mcvol1[0:2]))
print("    MC Roche lobe volume 2: %6.4f +/- %6.4f" % (mcvol2[0:2]))
print("    MC effective radius 1: %6.4f +/- %6.4f" % (mcvol1[2:]))
print("    MC effective radius 2: %6.4f +/- %6.4f" % (mcvol2[2:]))

plt.contour(p, [l5pot*1.02, l3pot, l2pot, l1pot], colors=['g', 'c', 'b', 'r'], extent=[-1.5, 2, -1.6, 1.6])
plt.text(l1, 0, 'L1', horizontalalignment='center')
plt.text(l2, 0, 'L2', horizontalalignment='center')
plt.text(l3, 0, 'L3', horizontalalignment='center')
plt.text(l4[0], l4[1], 'L4', horizontalalignment='center')
plt.text(l5[0], l5[1], 'L5', horizontalalignment='center')
plt.show()

In [ ]:
from __future__ import print_function, division
from PyAstronomy import pyasl
import numpy as np
import matplotlib.pylab as plt

x, y = np.linspace(-1.5, 2, 300), np.linspace(-1.6, 1.6, 300)
xx, yy = np.meshgrid(x, y)
# Coordinates in orbital plain
z = 0

# Mass ratio
q = 0.2

# Get dimensional values of Roche potential
p = pyasl.rochepot_dl(xx, yy, z, q)

# Positions (and potentials) of Lagrange points
l1, l1pot = pyasl.get_lagrange_1(q)
l2, l2pot = pyasl.get_lagrange_2(q)
l3, l3pot = pyasl.get_lagrange_3(q)
l4, l5 = pyasl.get_lagrange_4(), pyasl.get_lagrange_5()
l4pot = pyasl.rochepot_dl(l4[0], l4[1], l4[2], q)
l5pot = pyasl.rochepot_dl(l5[0], l5[1], l5[2], q)


print("Effective (dimensionless) radii of first and second mass")
print("According to the approximation of Eggleton 1983:")
r1eff = pyasl.roche_lobe_radius_eggleton(q, 1)
r2eff = pyasl.roche_lobe_radius_eggleton(q, 2)
print("    Reff1: %5.3f" % r1eff)
print("    Reff2: %5.3f" % r2eff)
print()
print("Roche volume and effective radius from Monte Carlo integration:")
mcvol1 = pyasl.roche_vol_MC(q, 1)
mcvol2 = pyasl.roche_vol_MC(q, 2)
print("    MC Roche lobe volume 1: %6.4f +/- %6.4f" % (mcvol1[0:2]))
print("    MC Roche lobe volume 2: %6.4f +/- %6.4f" % (mcvol2[0:2]))
print("    MC effective radius 1: %6.4f +/- %6.4f" % (mcvol1[2:]))
print("    MC effective radius 2: %6.4f +/- %6.4f" % (mcvol2[2:]))

plt.contour(p, [l5pot*1.02, l3pot, l2pot, l1pot], colors=['g', 'c', 'b', 'r'], extent=[-1.5, 2, -1.6, 1.6])
plt.text(l1, 0, 'L1', horizontalalignment='center')
plt.text(l2, 0, 'L2', horizontalalignment='center')
plt.text(l3, 0, 'L3', horizontalalignment='center')
plt.text(l4[0], l4[1], 'L4', horizontalalignment='center')
plt.text(l5[0], l5[1], 'L5', horizontalalignment='center')
plt.show()

In [ ]:
# %matplotlib notebook
from scipy.optimize import newton
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.path as mpath

In [ ]:
# This code defines a nice shape for the center of mass of the system.
circle = mpath.Path.unit_circle()
wedge_1 = mpath.Path.wedge(90, 180)
wedge_2 = mpath.Path.wedge(270, 0)

verts = np.concatenate([circle.vertices, wedge_1.vertices[::-1, ...], wedge_2.vertices[::-1, ...]])
codes = np.concatenate([circle.codes, wedge_1.codes, wedge_2.codes])
center_of_mass = mpath.Path(verts, codes)

In [ ]:
# These masses represent the Earth-Moon system
m_1 = 5.974E24  # kg
m_2 = 7.348E22 # kg
pi_2 = m_2/(m_1 + m_2)

# These give us the coordinates of the orbits of m_2 and m_1
x_2 = (1 - pi_2) * np.cos(np.linspace(0, np.pi, 100))
y_2 = (1 - pi_2) * np.sin(np.linspace(0, np.pi, 100))
x_1 = (-pi_2) * np.cos(np.linspace(0, np.pi, 100))
y_1 = (-pi_2) * np.sin(np.linspace(0, np.pi, 100))

In [ ]:
def collinear_lagrange(xstar, pi_2):
    """Calculate the resultant of the collinear Lagrange point equation.

    This is a function f(xstar, pi_2), where xstar is the nondimensional x coordinate
    and pi_2 is the nondimensional mass ratio. The function should be passed to
    scipy.optimize.newton (or another Newton solver) to find a value for xstar
    that satsifies the equation, for a given value of pi_2.

    The solver will try different values of xstar until the return value is equal to zero.
    """
    first_term = xstar
    second_term = (1 - pi_2) / np.abs(xstar + pi_2)**3 * (xstar + pi_2)
    third_term = pi_2 / np.abs(xstar - 1 + pi_2)**3 * (xstar - 1 + pi_2)
    return first_term - second_term - third_term


In [ ]:
def collinear_lagrange(xstar, pi_2):
    """Calculate the resultant of the collinear Lagrange point equation.

    This is a function f(xstar, pi_2), where xstar is the nondimensional x coordinate
    and pi_2 is the nondimensional mass ratio. The function should be passed to
    scipy.optimize.newton (or another Newton solver) to find a value for xstar
    that satsifies the equation, for a given value of pi_2.

    The solver will try different values of xstar until the return value is equal to zero.
    """
    first_term = xstar
    second_term = (1 - pi_2) / np.abs(xstar + pi_2)**3 * (xstar + pi_2)
    third_term = pi_2 / np.abs(xstar - 1 + pi_2)**3 * (xstar - 1 + pi_2)
    return first_term - second_term - third_term

In [ ]:
L_2 = newton(func=collinear_lagrange, x0=1, args=(pi_2,))
L_1 = newton(func=collinear_lagrange, x0=0, args=(pi_2,))
L_3 = newton(func=collinear_lagrange, x0=-1, args=(pi_2,))
print(f"{L_1=}, {L_2=}, {L_3=}")

In [ ]:
x_2 = (1 - pi_2) * np.cos(np.linspace(0, np.pi, 100))
y_2 = (1 - pi_2) * np.sin(np.linspace(0, np.pi, 100))
x_1 = (-pi_2) * np.cos(np.linspace(0, np.pi, 100))
y_1 = (-pi_2) * np.sin(np.linspace(0, np.pi, 100))

In [ ]:
fig, ax = plt.subplots(figsize=(5,5), dpi=96)
ax.set_xlabel("$x^*$")
ax.set_ylabel("$y^*$")

# Plot the orbits
ax.axhline(0, color='k')
ax.plot(np.hstack((x_2, x_2[::-1])), np.hstack((y_2, -y_2[::-1])))
ax.plot(np.hstack((x_1, x_1[::-1])), np.hstack((y_1, -y_1[::-1])))
ax.plot([-pi_2, 0.5 - pi_2, 1 - pi_2, 0.5 - pi_2, -pi_2], [0, np.sqrt(3)/2, 0, -np.sqrt(3)/2, 0], 'k', ls="--", lw=1)

# Plot the Lagrange Points and masses
ax.plot(L_1, 0, 'rv', label="$L_1$")
ax.plot(L_2, 0, 'r^', label="$L_2$")
ax.plot(L_3, 0, 'rp', label="$L_3$")
ax.plot(0.5 - pi_2, np.sqrt(3)/2, 'rX', label="$L_4$")
ax.plot(0.5 - pi_2, -np.sqrt(3)/2, 'rs', label="$L_5$")
ax.plot(0, 0, 'k', marker=center_of_mass, markersize=10)
ax.plot(-pi_2, 0, 'bo', label="$m_1$")
ax.plot(1 - pi_2, 0, 'go', label="$m_2$")
ax.legend(loc='upper right', bbox_to_anchor=(1.25, 1.0))
ax.set_aspect("equal")

In [ ]:
import pandas as pd
import plotly.express as px

df = pd.DataFrame(dict(
    x = [-pi_2, 0.5- pi_2, 1 - pi_2, 0.5 - pi_2, -pi_2],
    y = [0, np.sqrt(3)/2, 0, -np.sqrt(3)/2, 0]))
df.x



In [ ]:
import plotly.graph_objs as go
import plotly
import pandas as pd
#import numpy as np

# This code defines a nice shape for the center of mass of the system.
circle = mpath.Path.unit_circle()
wedge_1 = mpath.Path.wedge(90, 180)
wedge_2 = mpath.Path.wedge(270, 0)

verts = np.concatenate([circle.vertices, wedge_1.vertices[::-1, ...], wedge_2.vertices[::-1, ...]])
codes = np.concatenate([circle.codes, wedge_1.codes, wedge_2.codes])
center_of_mass = mpath.Path(verts, codes)

# These masses represent the Earth-Moon system
m_1 = 5.974E24  # kg
m_2 = 7.348E22 # kg
pi_2 = m_2/(m_1 + m_2)

# These give us the coordinates of the orbits of m_2 and m_1
x_2 = (1 - pi_2) * np.cos(np.linspace(0, np.pi, 100))
y_2 = (1 - pi_2) * np.sin(np.linspace(0, np.pi, 100))
x_1 = (-pi_2) * np.cos(np.linspace(0, np.pi, 100))
y_1 = (-pi_2) * np.sin(np.linspace(0, np.pi, 100))

def collinear_lagrange(xstar, pi_2):
    """Calculate the resultant of the collinear Lagrange point equation.

    This is a function f(xstar, pi_2), where xstar is the nondimensional x coordinate
    and pi_2 is the nondimensional mass ratio. The function should be passed to
    scipy.optimize.newton (or another Newton solver) to find a value for xstar
    that satsifies the equation, for a given value of pi_2.

    The solver will try different values of xstar until the return value is equal to zero.
    """
    first_term = xstar
    second_term = (1 - pi_2) / np.abs(xstar + pi_2)**3 * (xstar + pi_2)
    third_term = pi_2 / np.abs(xstar - 1 + pi_2)**3 * (xstar - 1 + pi_2)
    return first_term - second_term - third_term

def collinear_lagrange(xstar, pi_2):
    """Calculate the resultant of the collinear Lagrange point equation.

    This is a function f(xstar, pi_2), where xstar is the nondimensional x coordinate
    and pi_2 is the nondimensional mass ratio. The function should be passed to
    scipy.optimize.newton (or another Newton solver) to find a value for xstar
    that satsifies the equation, for a given value of pi_2.

    The solver will try different values of xstar until the return value is equal to zero.
    """
    first_term = xstar
    second_term = (1 - pi_2) / np.abs(xstar + pi_2)**3 * (xstar + pi_2)
    third_term = pi_2 / np.abs(xstar - 1 + pi_2)**3 * (xstar - 1 + pi_2)
    return first_term - second_term - third_term

L_2 = newton(func=collinear_lagrange, x0=1, args=(pi_2,))
L_1 = newton(func=collinear_lagrange, x0=0, args=(pi_2,))
L_3 = newton(func=collinear_lagrange, x0=-1, args=(pi_2,))
print(f"{L_1=}, {L_2=}, {L_3=}")

x_2 = (1 - pi_2) * np.cos(np.linspace(0, np.pi, 100))
y_2 = (1 - pi_2) * np.sin(np.linspace(0, np.pi, 100))
x_1 = (-pi_2) * np.cos(np.linspace(0, np.pi, 100))
y_1 = (-pi_2) * np.sin(np.linspace(0, np.pi, 100))


df = pd.DataFrame(dict(
    x = [-pi_2, 0.5- pi_2, 1 - pi_2, 0.5 - pi_2, -pi_2],
    y = [0, np.sqrt(3)/2, 0, -np.sqrt(3)/2, 0]))

x_2 = (1 - pi_2) * np.cos(np.linspace(0, np.pi, 100))
y_2 = (1 - pi_2) * np.sin(np.linspace(0, np.pi, 100))

fig = go.Figure(go.Scatter(x = df.x, y = df.y,
    mode = 'lines',
    line = dict(shape = 'linear', color = 'rgb(10, 120, 24)', dash = 'dot'),
    name='None'))

fig.add_trace(go.Scatter(x = (-1.25,1.3), y = (0,0), 
    marker=dict(size=6, color='navy'),
    mode='lines',  name='Horz Axis')) 

fig.add_trace(go.Scatter(x = np.hstack((x_2, x_2[::-1])), 
                         y = np.hstack((y_2, -y_2[::-1])), 
    marker=dict(size=4, color='dodgerblue'),
    mode='lines',  name='S-Curve'))

fig.add_trace(go.Scatter(x = (x_2[33],x_2[33]), y = (np.sqrt(3)/2,np.sqrt(3)/2),
                         name='L4', marker=dict(size=18, color='blueviolet')))

fig.add_trace(go.Scatter(x = (x_2[33],x_2[33]),  
                         y = (-np.sqrt(3)/2, -np.sqrt(3)/2),
                         name='L5', marker=dict(size=18, color='limegreen')))

fig.add_trace(go.Scatter(x = (L_1,L_1), y = (0,0),
                         name='L1', marker=dict(size=18, color='red')))

fig.add_trace(go.Scatter(x = (L_2,L_2), y = (0,0),
                         name='L2', marker=dict(size=18, color='darkred'))) 

fig.add_trace(go.Scatter(x = (L_3,L_3), y = (0,0),
                         name='L3', marker=dict(size=18, color='darkturquoise')))

fig.add_trace(go.Scatter(x = (0,0), y = (0,0),
                         name='m1',  marker=dict(size=14, color='fuchsia', symbol='diamond'))) 

fig.add_trace(go.Scatter(y = (0,0), x = (1-pi_2,1-pi_2),
                         name='m2', marker=dict(size=14, color='blue', symbol='diamond'))) 
    
fig.update_yaxes(scaleanchor = "x", scaleratio = 1)

fig.update_layout(autosize=False, font_size=18,
    width=700, height=600,
    margin=dict(l=25, r=100, b=25, t=50, pad=10),
    paper_bgcolor='white',
    legend_font=dict(family="Arial", size=16,
        color="RebeccaPurple"
    ))

fig.show()

In [ ]:
x_2[33]

In [ ]:
# These masses represent the Earth-Moon system
m_1 = 5.974E24  # kg
m_2 = 7.348E22 # kg
pi_2 = m_2/(m_1 + m_2)


In [ ]:
x_0 = 1 - pi_2
y_0 = .0455
z_0 = 0
vx_0 = -0.5
vy_0 = 0.5
vz_0 = 0

# Then stack everything together into the state vector
r_0 = np.array((x_0, y_0, z_0))
v_0 = np.array((vx_0, vy_0, vz_0))
Y_0 = np.hstack((r_0, v_0))


In [ ]:
def nondim_cr3bp(t, Y):
    """Solve the CR3BP in nondimensional coordinates.
    
    The state vector is Y, with the first three components as the
    position of $m$, and the second three components its velocity.
    
    The solution is parameterized on $\\pi_2$, the mass ratio.
    """
    # Get the position and velocity from the solution vector
    x, y, z = Y[:3]
    xdot, ydot, zdot = Y[3:]

    # Define the derivative vector
    Ydot = np.zeros_like(Y)
    Ydot[:3] = Y[3:]

    sigma = np.sqrt(np.sum(np.square([x + pi_2, y, z])))
    psi = np.sqrt(np.sum(np.square([x - 1 + pi_2, y, z])))
    Ydot[3] = 2 * ydot + x - (1 - pi_2) * (x + pi_2) / sigma**3 - pi_2 * (x - 1 + pi_2) / psi**3
    Ydot[4] = -2 * xdot + y - (1 - pi_2) * y / sigma**3 - pi_2 * y / psi**3
    Ydot[5] = -(1 - pi_2)/sigma**3 * z - pi_2/psi**3 * z
    return Ydot


In [ ]:
t_0 = 0  # nondimensional time
t_f = 20  # nondimensional time
t_points = np.linspace(t_0, t_f, 1000)
sol = solve_ivp(nondim_cr3bp, [t_0, t_f], Y_0, t_eval=t_points)

Y = sol.y.T
r = Y[:, :3]  # nondimensional distance
v = Y[:, 3:]  # nondimensional velocity


In [ ]:
# Input 1
from scipy.integrate import solve_ivp
# These masses represent the Earth-Moon system
m_1 = 5.974E24  # kg
m_2 = 7.348E22 # kg
pi_2 = m_2/(m_1 + m_2)

#Input 2
x_0 = 1 - pi_2
y_0 = .0455
z_0 = 0
vx_0 = -0.5
vy_0 = 0.5
vz_0 = 0

# Then stack everything together into the state vector
r_0 = np.array((x_0, y_0, z_0))
v_0 = np.array((vx_0, vy_0, vz_0))
Y_0 = np.hstack((r_0, v_0))

#Input 3
def nondim_cr3bp(t, Y):
    """Solve the CR3BP in nondimensional coordinates.
    
    The state vector is Y, with the first three components as the
    position of $m$, and the second three components its velocity.
    
    The solution is parameterized on $\\pi_2$, the mass ratio.
    """
    # Get the position and velocity from the solution vector
    x, y, z = Y[:3]
    xdot, ydot, zdot = Y[3:]

    # Define the derivative vector
    Ydot = np.zeros_like(Y)
    Ydot[:3] = Y[3:]

    sigma = np.sqrt(np.sum(np.square([x + pi_2, y, z])))
    psi = np.sqrt(np.sum(np.square([x - 1 + pi_2, y, z])))
    Ydot[3] = 2 * ydot + x - (1 - pi_2) * (x + pi_2) / sigma**3 - pi_2 * (x - 1 + pi_2) / psi**3
    Ydot[4] = -2 * xdot + y - (1 - pi_2) * y / sigma**3 - pi_2 * y / psi**3
    Ydot[5] = -(1 - pi_2)/sigma**3 * z - pi_2/psi**3 * z
    return Ydot

#Input 4
t_0 = 0  # nondimensional time
t_f = 20  # nondimensional time
t_points = np.linspace(t_0, t_f, 1000)
sol = solve_ivp(nondim_cr3bp, [t_0, t_f], Y_0, t_eval=t_points)

Y = sol.y.T
r = Y[:, :3]  # nondimensional distance
v = Y[:, 3:]  # nondimensional velocity

x_2 = (1 - pi_2) * np.cos(np.linspace(0, np.pi, 100))
y_2 = (1 - pi_2) * np.sin(np.linspace(0, np.pi, 100))
fig, ax = plt.subplots(figsize=(5,5), dpi=96)

# Plot the orbits
ax.plot(r[:, 0], r[:, 1], 'r', label="Trajectory")
ax.axhline(0, color='k')
ax.plot(np.hstack((x_2, x_2[::-1])), np.hstack((y_2, -y_2[::-1])))
ax.plot(-pi_2, 0, 'bo', label="$m_1$")
ax.plot(1 - pi_2, 0, 'go', label="$m_2$")
ax.plot(x_0, y_0, 'ro')
ax.set_aspect("equal")


In [ ]:
import plotly.graph_objs as go
import plotly

# Input 1
from scipy.integrate import solve_ivp
# These masses represent the Earth-Moon system
m_1 = 5.974E24  # kg
m_2 = 7.348E22 # kg
pi_2 = m_2/(m_1 + m_2)

#Input 2
x_0 = 1 - pi_2
y_0 = .0455
z_0 = 0
vx_0 = -0.5
vy_0 = 0.5
vz_0 = 0

# Then stack everything together into the state vector
r_0 = np.array((x_0, y_0, z_0))
v_0 = np.array((vx_0, vy_0, vz_0))
Y_0 = np.hstack((r_0, v_0))

#Input 3
def nondim_cr3bp(t, Y):
    """Solve the CR3BP in nondimensional coordinates.
    
    The state vector is Y, with the first three components as the
    position of $m$, and the second three components its velocity.
    
    The solution is parameterized on $\\pi_2$, the mass ratio.
    """
    # Get the position and velocity from the solution vector
    x, y, z = Y[:3]
    xdot, ydot, zdot = Y[3:]

    # Define the derivative vector
    Ydot = np.zeros_like(Y)
    Ydot[:3] = Y[3:]

    sigma = np.sqrt(np.sum(np.square([x + pi_2, y, z])))
    psi = np.sqrt(np.sum(np.square([x - 1 + pi_2, y, z])))
    Ydot[3] = 2 * ydot + x - (1 - pi_2) * (x + pi_2) / sigma**3 - pi_2 * (x - 1 + pi_2) / psi**3
    Ydot[4] = -2 * xdot + y - (1 - pi_2) * y / sigma**3 - pi_2 * y / psi**3
    Ydot[5] = -(1 - pi_2)/sigma**3 * z - pi_2/psi**3 * z
    return Ydot

#Input 4
t_0 = 0  # nondimensional time
t_f = 20  # nondimensional time
t_points = np.linspace(t_0, t_f, 1000)
sol = solve_ivp(nondim_cr3bp, [t_0, t_f], Y_0, t_eval=t_points)

Y = sol.y.T
r = Y[:, :3]  # nondimensional distance
v = Y[:, 3:]  # nondimensional velocity

x_2 = (1 - pi_2) * np.cos(np.linspace(0, np.pi, 100))
y_2 = (1 - pi_2) * np.sin(np.linspace(0, np.pi, 100))

fig = go.Figure()

fig.add_trace(go.Scatter(
    x = (-1.1,1.2),
    y = (0,0), 
    marker=dict(size=6, color='navy'),
    mode='lines',  name='Horz Axis')) 

fig.add_trace(go.Scatter(
    x = np.hstack((x_2, x_2[::-1])), 
    y = np.hstack((y_2, -y_2[::-1])), 
    marker=dict(size=3, color='dodgerblue'),
    mode='lines',  name='S-Curve'))

fig.add_trace(go.Scatter(
    x = r[:, 0],
    y = r[:, 1], 
    marker=dict(size=6, color='red'),
    mode='lines', name='Trajectory')) 

fig.add_trace(go.Scatter(
    x = (0,0),
    y = (0,0), 
    marker=dict(size=10, color='blueviolet', symbol='square'),
    mode='markers', name='m1')) 

fig.add_trace(go.Scatter(
    y = (0,0),
    x = (1-pi_2,1-pi_2), 
    marker=dict(size=10, color='limegreen', symbol='square'),
    mode='markers',  name='m2'))

fig.add_trace(go.Scatter(
    x = (x_2[1],x_2[1]),
    y = (y_2[1],y_2[1]), 
    marker=dict(size=8, color='red'),
    mode='markers',  name='Initial Value')) 

fig.update_yaxes(
    scaleanchor = "x",
    scaleratio = 1)

fig.update_layout(
    autosize=False,
    font_size=18,
    width=600, height=500,
    margin=dict(l=25, r=100, b=25, t=50, pad=10),
    paper_bgcolor='white',
    legend_font=dict(
        family="Arial",
        size=16,
        color="RebeccaPurple"
    ))

fig.show()

In [ ]:
speed_sq = np.sum(np.square(v), axis=1)

r[:, 0] += pi_2
sigma = np.sqrt(np.sum(np.square(r), axis=1))
r[:, 0] -= 1.0
psi = np.sqrt(np.sum(np.square(r), axis=1))
r[:, 0] = r[:, 0] + 1.0 - pi_2

J = 0.5 * speed_sq - (1 - pi_2) / sigma - pi_2 / psi - 0.5 * ((1 - pi_2) * sigma**2 + pi_2 * psi**2)

In [ ]:
fig, ax = plt.subplots(figsize=(8,6), dpi=96)
ax.plot(sol.t, J, label="Jacobi Constant")
ax.axhline(J[0], color="C1", label="Initial Jacobi Constant")
ax.legend(loc="center left")
ax.set_xlabel("$\\tau$")
ax.set_ylabel("$J$")

In [ ]:
sol_hp = solve_ivp(nondim_cr3bp, [t_0, t_f], Y_0, atol=1e-9, rtol=1e-6, t_eval=t_points)
Y_hp = sol_hp.y.T
r_hp = Y_hp[:, :3]  # nondimensional distance
v_hp = Y_hp[:, 3:]  # nondimensional velocity

speed_sq_hp = np.sum(np.square(v_hp), axis=1)

r_hp[:, 0] += pi_2
sigma_hp = np.sqrt(np.sum(np.square(r_hp), axis=1))
r_hp[:, 0] -= 1
psi_hp = np.sqrt(np.sum(np.square(r_hp), axis=1))
r_hp[:, 0] = r_hp[:, 0] + 1 - pi_2

J_hp = 0.5 * speed_sq_hp - (1 - pi_2) / sigma_hp - pi_2 / psi_hp - 0.5 * ((1 - pi_2) * sigma_hp**2 + pi_2 * psi_hp**2)

fig, ax = plt.subplots(figsize=(6,4), dpi=96)
ax.plot(sol_hp.t, J_hp, label="Jacobi Constant")
ax.axhline(J_hp[0], color="C1", label="Initial Jacobi Constant")
ax.set_ylim(-1.4988, -1.49886)
ax.legend(loc="upper right")
ax.set_xlabel("$\\tau$")
ax.set_ylabel("$J$")
ax.ticklabel_format(style="plain", useOffset=False)

In [ ]:
import plotly.express as px
from plotly.subplots import make_subplots

fig = make_subplots()

fig.add_scatter(x = (0,20),
    y = (J_hp[0],J_hp[0]),
    name="Initial Jacobi Constant")

fig.add_scatter(x = sol_hp.t, y = J_hp, name="Jacobi Constant")  

fig.update_xaxes(title_text = "$\\tau$",
        title_font = {"size": 20}, title_standoff = 10)

fig.update_yaxes(title_text="$J$", title_font = {"size": 20}, 
                 title_standoff = 10)

fig.update_layout(autosize=True, font_size=16,
    width=750, height=450,
    margin=dict(l=10, r=10, b=10, t=20, pad=10),
    paper_bgcolor='snow', plot_bgcolor='white',
    legend_font=dict(family="Arial", size=18, color="darkblue"),
    legend=dict(yanchor="bottom", y=0, xanchor="left", x=0.05),
    xaxis = dict(tickmode='linear', tick0 = 0.0, dtick = 2.5)
)

fig.show()

In [ ]:
sol_hp = solve_ivp(nondim_cr3bp, [t_0, t_f], Y_0, atol=1e-8, rtol=1e-6, t_eval=t_points)
Y_hp = sol_hp.y.T
r_hp = Y_hp[:, :3]  # nondimensional distance
v_hp = Y_hp[:, 3:]  # nondimensional velocity

speed_sq_hp = np.sum(np.square(v_hp), axis=1)

r_hp[:, 0] += pi_2
sigma_hp = np.sqrt(np.sum(np.square(r_hp), axis=1))
r_hp[:, 0] -= 1
psi_hp = np.sqrt(np.sum(np.square(r_hp), axis=1))
r_hp[:, 0] = r_hp[:, 0] + 1 - pi_2

J_hp = 0.5 * speed_sq_hp - (1 - pi_2) / sigma_hp - pi_2 / psi_hp - 0.5 * ((1 - pi_2) * sigma_hp**2 + pi_2 * psi_hp**2)

fig, ax = plt.subplots(figsize=(6,4), dpi=96)
ax.plot(sol_hp.t, J_hp, label="Jacobi Constant")
ax.axhline(J_hp[0], color="C1", label="Initial Jacobi Constant")
ax.set_ylim(-1.4988, -1.49886)
ax.legend(loc="upper right")
ax.set_xlabel("$\\tau$")
ax.set_ylabel("$J$")
ax.ticklabel_format(style="plain", useOffset=False)

In [ ]:
sol_hp2 = solve_ivp(nondim_cr3bp, [t_0, t_f], Y_0, atol=1e-9, rtol=1e-8, t_eval=t_points)
Y_hp2 = sol_hp2.y.T
r_hp2 = Y_hp2[:, :3]  # nondimensional distance
v_hp2 = Y_hp2[:, 3:]  # nondimensional velocity

speed_sq_hp2 = np.sum(np.square(v_hp2), axis=1)

r_hp2[:, 0] += pi_2
sigma_hp2 = np.sqrt(np.sum(np.square(r_hp2), axis=1))
r_hp2[:, 0] -= 1
psi_hp2 = np.sqrt(np.sum(np.square(r_hp2), axis=1))
r_hp2[:, 0] = r_hp2[:, 0] + 1 - pi_2

J_hp2 = 0.5 * speed_sq_hp2 - (1 - pi_2) / sigma_hp2 - pi_2 / psi_hp2 - 0.5 * ((1 - pi_2) * sigma_hp2**2 + pi_2 * psi_hp2**2)

# Plot

fig2 = make_subplots()

fig2.add_scatter(x = sol_hp2.t, y = J_hp2, 
                 name="Jacobi Constant")  

fig2.add_scatter(x = (0,20), y = (J_hp2[0],J_hp2[0]),
    name="Initial Jacobi Constant", mode='lines',
    marker=dict(size=12, color='magenta'))

fig2.update_xaxes(title_text = "$\\tau$", 
                  title_font = {"size": 20},
        title_standoff = 10)

fig2.update_yaxes(title_text="$J$", title_font = {"size": 20}, 
                 title_standoff = 10)

fig2.update_layout(
    autosize=True, font_size=16, width=750, height=350,
    margin=dict(l=10, r=10, b=10, t=20, pad=10),
    paper_bgcolor='snow', plot_bgcolor='white',
    legend_font=dict(family="Arial", size=18, color="navy"),
    legend=dict(yanchor="top", y=.99, xanchor="left", x=0.05),
    xaxis = dict(tickmode='linear', tick0 = 0.0, dtick = 2.5)
)

fig2.show()

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5), dpi=96)

# Plot the orbits
ax.plot(r_hp[:, 0], r[:, 1], 'r', label="Reduced Tolerances Trajectory")
ax.axhline(0, color='k')
ax.plot(np.hstack((x_2, x_2[::-1])), np.hstack((y_2, -y_2[::-1])))
ax.plot(-pi_2, 0, 'bo')
ax.plot(1 - pi_2, 0, 'go')
ax.plot(x_0, y_0, 'ro')
ax.plot(r[:, 0], r[:, 1], 'C2', label="Original Trajectory")
ax.set_aspect("equal")

In [ ]:
import plotly.graph_objs as go
import plotly

x_2 = (1 - pi_2) * np.cos(np.linspace(0, np.pi, 100))
y_2 = (1 - pi_2) * np.sin(np.linspace(0, np.pi, 100))

fig = go.Figure()

fig.add_trace(go.Scatter(x = (-1.1,1.2), y = (0,0), 
    marker=dict(size=6, color='navy'),
    mode='lines',  name='Horz Axis')) 

fig.add_trace(go.Scatter(x = np.hstack((x_2, x_2[::-1])), 
    y = np.hstack((y_2, -y_2[::-1])), 
    marker=dict(size=3, color='dodgerblue'),
    mode='lines',  name='S-Curve'))

fig.add_trace(go.Scatter(x = r[:, 0], y = r[:, 1], 
    marker=dict(size=6, color='red'),
    mode='lines', name='Original Trajectory')) 

fig.add_trace(go.Scatter(x = r_hp[:, 0], y = r[:, 1], 
    marker=dict(size=6, color='green'),
    mode='lines', name='Reduced Tolerances Trajectory')) 

fig.add_trace(go.Scatter(x = (0,0), y = (0,0), 
    marker=dict(size=10, color='blueviolet', symbol='square'),
    mode='markers', name='m 1')) 

fig.add_trace(go.Scatter(y = (0,0), x = (1-pi_2,1-pi_2), 
    marker=dict(size=10, color='limegreen', symbol='square'),
    mode='markers',  name='m 2'))

fig.add_trace(go.Scatter(x = (x_2[1],x_2[1]), y = (y_2[1],y_2[1]), 
    marker=dict(size=10, color='red'),
    mode='markers',  name='Initial Value')) 

fig.update_yaxes(scaleanchor = "x", scaleratio = 1)

fig.update_layout(autosize=False, font_size=18,
    width=800, height=500,
    margin=dict(l=25, r=100, b=25, t=50, pad=10),
    paper_bgcolor='white',
    legend_font=dict(family="Arial", size=18,
        color="RebeccaPurple"))

fig.show()

In [ ]:
fig, J_ax = plt.subplots(figsize=(6, 4), dpi=96)
Jacobi_plot = J_ax.plot(sol_hp.t, J_hp, label="Jacobi Constant")
initial_Jacobi = J_ax.axhline(J_hp[0], color='C1', label="Initial Jacobi Constant")
s_ax = J_ax.twinx()
sigma_plot = s_ax.plot(sol_hp.t, sigma_hp, 'C2', label="$\sigma$")
s_ax.set_ylim(0, 1.15)
J_ax.ticklabel_format(style="plain", useOffset=False)
J_ax.set_xlabel("$\\tau$")
J_ax.set_ylabel("$J$")
s_ax.set_ylabel("$\\sigma$")
fig.savefig("plot01.png", dpi=300, format='png')


In [ ]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots

fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Scatter(x = sol_hp.t, y = J_hp, 
    marker=dict(size=6, color='navy'),
    mode='lines',  name="Jacobi Constant"),
    secondary_y=False,
)  

fig.add_trace(go.Scatter(x = sol_hp.t, y = sigma_hp,
    marker=dict(size=6, color='red'),
    mode='lines',  name="sigma"),
    secondary_y=True,
) 

fig.add_trace(go.Scatter(x = (0,20), y = (J_hp[0],J_hp[0]), 
    marker=dict(size=6, color='green'),
    mode = 'lines',  name = 'Initial Jacobi Constant')) 

fig.update_xaxes(title_text = "tau",
        title_font = {"size": 20}, title_standoff = 10)

fig.update_yaxes(title_text="J", 
                 secondary_y = False, title_standoff = 10)
fig.update_yaxes(title_text = "secondary sigma", 
                 secondary_y = True, title_standoff = 10)

fig.update_layout(autosize=True, font_size=18,
    width=900, height=400,
    margin=dict(l=10, r=10, b=10, t=20, pad=10),
    paper_bgcolor='papayawhip',
    plot_bgcolor='white',
    legend_font=dict(family="Arial", size=18, color="Purple"),
    xaxis = dict(tickmode ='linear', tick0=0.0, dtick=2.5))

fig.show()

In [ ]:
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objs as go
import plotly
from scipy.integrate import solve_ivp
import numpy as np
#import matplotlib.pyplot as plt

# INPUT 2 41 (Python)

# These masses represent the Earth-Moon system
m_1 = 5.974E24  # kg.
m_2 = 7.348E22 # kg.
pi_2 = m_2/(m_1 + m_2)


# INPUT 2 42 (Python)

x_0  =  1 - pi_2
y_0  =  0.0455
z_0  =  0.0
vx_0 = -0.5
vy_0 =  0.5
vz_0 =  0.0

# Then stack everything together into the state vector.
r_0 = np.array((x_0, y_0, z_0))
v_0 = np.array((vx_0, vy_0, vz_0))
Y_0 = np.hstack((r_0, v_0))


# INPUT 2 43 (Python)

def nondim_cr3bp(t, Y):
    """Solve the CR3BP in nondimensional coordinates.
    
    The state vector is Y, with the first three components as the
    position of $m$, and the second three components its velocity.
    
    The solution is parameterized on $\\pi_2$, the mass ratio.
    """
    # Get the position and velocity from the solution vector
    x, y, z = Y[:3]
    xdot, ydot, zdot = Y[3:]

    # Define the derivative vector.
    Ydot = np.zeros_like(Y)
    Ydot[:3] = Y[3:]

    sigma = np.sqrt(np.sum(np.square([x + pi_2, y, z])))
    psi = np.sqrt(np.sum(np.square([x - 1 + pi_2, y, z])))
    Ydot[3] = 2 * ydot + x - (1 - pi_2) * (x + pi_2) / sigma**3 - pi_2 * (x - 1 + pi_2) / psi**3
    Ydot[4] = -2 * xdot + y - (1 - pi_2) * y / sigma**3 - pi_2 * y / psi**3
    Ydot[5] = -(1 - pi_2)/sigma**3 * z - pi_2/psi**3 * z
    return Ydot

#INPUT 2 44 (Python)

t_0 = 0  # nondimensional time.
t_f = 20  # nondimensional time.
t_points = np.linspace(t_0, t_f, 1000)
sol = solve_ivp(nondim_cr3bp, [t_0, t_f], Y_0, t_eval=t_points)

Y = sol.y.T
r = Y[:, :3]  # nondimensional distance.
v = Y[:, 3:]  # nondimensional velocity.


# INPUT 2 45 (Python)

x_2 = (1 - pi_2) * np.cos(np.linspace(0, np.pi, 100))
y_2 = (1 - pi_2) * np.sin(np.linspace(0, np.pi, 100))

# INPUT 2 46 (Python)

r_hp[:, 0] += pi_2
sigma_hp = np.sqrt(np.sum(np.square(r_hp), axis=1))
r_hp[:, 0] -= 1
psi_hp = np.sqrt(np.sum(np.square(r_hp), axis=1))
r_hp[:, 0] = r_hp[:, 0] + 1 - pi_2

sol_hp = solve_ivp(nondim_cr3bp, [t_0, t_f], Y_0, atol=1e-9, rtol=1e-6, t_eval=t_points)

speed_sq_hp = np.sum(np.square(v_hp), axis=1)
sigma_hp = np.sqrt(np.sum(np.square(r_hp), axis=1))
J_hp = 0.5 * speed_sq_hp - (1 - pi_2) / sigma_hp - pi_2 / psi_hp - 0.5 * ((1 - pi_2) * sigma_hp**2 + pi_2 * psi_hp**2)


speed_sq = np.sum(np.square(v), axis=1)

r[:, 0] += pi_2
sigma = np.sqrt(np.sum(np.square(r), axis=1))
r[:, 0] -= 1.0
psi = np.sqrt(np.sum(np.square(r), axis=1))
r[:, 0] = r[:, 0] + 1.0 - pi_2

J = 0.5 * speed_sq - (1 - pi_2) / sigma - pi_2 / psi - 0.5 * ((1 - pi_2) * sigma**2 + pi_2 * psi**2)

sol_hp = solve_ivp(nondim_cr3bp, [t_0, t_f], Y_0, atol=1e-9, rtol=1e-6, t_eval=t_points)

# INPUT 2 48 (Python)

sol_hp2 = solve_ivp(nondim_cr3bp, [t_0, t_f], Y_0, atol=1e-9, 
rtol=1e-8, t_eval=t_points)
Y_hp2 = sol_hp2.y.T
r_hp2 = Y_hp2[:, :3]  # nondimensional distance.
v_hp2 = Y_hp2[:, 3:]  # nondimensional velocity.

speed_sq_hp2 = np.sum(np.square(v_hp2), axis=1)

r_hp2[:, 0] += pi_2
sigma_hp2 = np.sqrt(np.sum(np.square(r_hp2), axis=1))
r_hp2[:, 0] -= 1
psi_hp2 = np.sqrt(np.sum(np.square(r_hp2), axis=1))
r_hp2[:, 0] = r_hp2[:, 0] + 1 - pi_2

J_hp2 = 0.5 * speed_sq_hp2 - (1 - pi_2) / sigma_hp2 - pi_2 / psi_hp2 - 0.5 * ((1 - pi_2) * sigma_hp2**2 + pi_2 * psi_hp2**2)




fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_scatter(x = (0,20),
    y = (J_hp[0],J_hp[0]),
    name="Initial Jacobi Constant")

fig.add_scatter(x = sol_hp.t, y = J_hp,
    name="Jacobi Constant",
    secondary_y=False,)  

fig.add_scatter(x = sol_hp.t, y = sigma_hp,
    name="sigma",
    secondary_y=True,) 

fig.update_xaxes(
        title_text = "tau",
        title_font = {"size": 20},
        title_standoff = 10)

fig.update_yaxes(title_text="J", secondary_y=False, title_standoff = 10)
fig.update_yaxes(title_text="<sigma", secondary_y=True, title_standoff = 10)

fig.update_layout(
    autosize=True,
    font_size=16,
    width=750, height=450,
    margin=dict(l=10, r=10, b=10, t=20, pad=10),
    paper_bgcolor='snow',
    plot_bgcolor='white',
    legend_font=dict(
        family="Arial",
        size=18,
        color="darkblue"),
    legend=dict(
        yanchor="top",
        y=1.35,
        xanchor="right",
        x=0.99),
    xaxis = dict(
        tickmode = 'linear',
        tick0 = 0.0,
        dtick = 2.5)
)

fig.show()


In [ ]:
import numpy as np
import scipy
from scipy.integrate import odeint
from mpl_toolkits.mplot3d import Axes3D
from scipy.optimize import brentq
from scipy.optimize import fsolve
import pylab

me = 5.974 * 10 ** 24  #  mass of the earth
mm = 7.348 * 10 ** 22  #  mass of the moon
G = 6.67259 * 10 ** -20  #  gravitational parameter
re = 6378.0  #  radius of the earth in km
rm = 1737.0  #  radius of the moon in km
r12 = 384400.0  #  distance between the CoM of the earth and moon
M = me + mm
d = 200  #  distance the spacecraft is above the Earth
pi2 = mm / M
pi1 = me / M
mue = 398600.0  #  gravitational parameter of earth km^3/sec^2
mum = G * mm  #  grav param of the moon
omega = np.sqrt(mue / r12 ** 3)
vbo = 10.9148
nu = -np.pi*0.5
gamma = 20*np.pi/180.0  #  angle in radians of the flight path

vx = vbo * (np.sin(gamma) * np.cos(nu) - np.cos(gamma) * np.sin(nu))
#  velocity of the bo in the x direction
vy = vbo * (np.sin(gamma) * np.sin(nu) + np.cos(gamma) * np.cos(nu))
#  velocity of the bo in the y direction

xrel = (re + d)*np.cos(nu)-pi2*r12
#  spacecraft x location relative to the earth
yrel = (re + 200.0) * np.sin(nu)

u0 = [xrel, yrel, 0, vx, vy, 0]

def deriv(u, dt):
    return [u[3],  #  dotu[0] = u[3]
            u[4],  #  dotu[1] = u[4]
            u[5],  #  dotu[2] = u[5]
            (2 * omega * u[4] + omega ** 2 * u[0] - mue * (u[0] + pi2 * r12) /
             np.sqrt(((u[0] + pi2 * r12) ** 2 + u[1] ** 2) ** 3) - mum *
             (u[0] - pi1 * r12) /
             np.sqrt(((u[0] - pi1 * r12) ** 2 + u[1] ** 2) ** 3)),
            #  dotu[3] = that
            (-2 * omega * u[3] + omega ** 2 * u[1] - mue * u[1] /
             np.sqrt(((u[0] + pi2 * r12) ** 2 + u[1] ** 2) ** 3) - mum * u[1] /
             np.sqrt(((u[0] - pi1 * r12) ** 2 + u[1] ** 2) ** 3)),
            #  dotu[4] = that
            0]  #  dotu[5] = 0

dt = np.linspace(0, 535000, 535000)  #  secs to run the simulation
u = odeint(deriv, u0, dt)
x, y, z, x2, y2, z2 = u.T

# 3d plot
fig = plt.figure(figsize=(7, 7))
#fig = pylab.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot(x, y, z, color = 'r')

# 2d plot if you want
#fig2 = pylab.figure(2)
#ax2 = fig2.add_subplot(111)
#ax2.plot(x, y, color = 'r')

#  adding the moon
phi = np.linspace(0, 2 * np.pi, 100)
theta = np.linspace(0, np.pi, 100)
xm = rm * np.outer(np.cos(phi), np.sin(theta)) + r12 - pi2 * r12
ym = rm * np.outer(np.sin(phi), np.sin(theta))
zm = rm * np.outer(np.ones(np.size(phi)), np.cos(theta))
ax.plot_surface(xm, ym, zm, color = '#696969', linewidth = 0)
ax.auto_scale_xyz([-8000, 385000], [-8000, 385000], [-8000, 385000])
#  adding the earth
xe = re * np.outer(np.cos(phi), np.sin(theta)) - pi2 * r12
ye = re * np.outer(np.sin(phi), np.sin(theta))
ze = re * np.outer(np.ones(np.size(phi)), np.cos(theta))
ax.plot_surface(xe, ye, ze, color = '#4169E1', linewidth = 0)
ax.auto_scale_xyz([-8000, 385000], [-8000, 385000], [-8000, 385000])

ax.set_xlim3d(-20000, 385000)
ax.set_ylim3d(-20000, 80000)
ax.set_zlim3d(-50000, 50000)

pylab.show()

fig.savefig("plot02.png", dpi=300, format='png')

In [ ]:
z

In [ ]:
import plotly.graph_objs as go
import plotly
import numpy as np
import scipy
from scipy.integrate import odeint
from scipy.optimize import brentq
from scipy.optimize import fsolve
import pylab

me = 5.974 * 10 ** 24  #  mass of the earth
mm = 7.348 * 10 ** 22  #  mass of the moon
G = 6.67259 * 10 ** -20  #  gravitational parameter
re = 6378.0  #  radius of the earth in km
rm = 1737.0  #  radius of the moon in km
r12 = 384400.0  #  distance between the CoM of the earth and moon
M = me + mm
d = 200  #  distance the spacecraft is above the Earth
pi2 = mm / M
pi1 = me / M
mue = 398600.0  #  gravitational parameter of earth km^3/sec^2
mum = G * mm  #  grav param of the moon
omega = np.sqrt(mue / r12 ** 3)
vbo = 10.9148
nu = -np.pi*0.5
gamma = 20*np.pi/180.0  #  angle in radians of the flight path

vx = vbo * (np.sin(gamma) * np.cos(nu) - np.cos(gamma) * np.sin(nu))
#  velocity of the bo in the x direction
vy = vbo * (np.sin(gamma) * np.sin(nu) + np.cos(gamma) * np.cos(nu))
#  velocity of the bo in the y direction

xrel = (re + d)*np.cos(nu)-pi2*r12
#  spacecraft x location relative to the earth
yrel = (re + 200.0) * np.sin(nu)

u0 = [xrel, yrel, 0, vx, vy, 0]

def deriv(u, dt):
    return [u[3],  #  dotu[0] = u[3]
            u[4],  #  dotu[1] = u[4]
            u[5],  #  dotu[2] = u[5]
            (2 * omega * u[4] + omega ** 2 * u[0] - mue * (u[0] + pi2 * r12) /
             np.sqrt(((u[0] + pi2 * r12) ** 2 + u[1] ** 2) ** 3) - mum *
             (u[0] - pi1 * r12) /
             np.sqrt(((u[0] - pi1 * r12) ** 2 + u[1] ** 2) ** 3)),
            #  dotu[3] = that
            (-2 * omega * u[3] + omega ** 2 * u[1] - mue * u[1] /
             np.sqrt(((u[0] + pi2 * r12) ** 2 + u[1] ** 2) ** 3) - mum * u[1] /
             np.sqrt(((u[0] - pi1 * r12) ** 2 + u[1] ** 2) ** 3)),
            #  dotu[4] = that
            0]  #  dotu[5] = 0

dt = np.linspace(0, 535000, 535000)  #  secs to run the simulation
u = odeint(deriv, u0, dt)
x, y, z, x2, y2, z2 = u.T
phi = np.linspace(0, 2 * np.pi, 100)
theta = np.linspace(0, np.pi, 100)

# 3d plot

#  adding the moon
xm = rm * np.outer(np.cos(phi), np.sin(theta)) + r12 - pi2 * r12
ym = rm * np.outer(np.sin(phi), np.sin(theta))
zm = rm * np.outer(np.ones(np.size(phi)), np.cos(theta))

#  adding the earth
xe = re * np.outer(np.cos(phi), np.sin(theta)) - pi2 * r12
ye = re * np.outer(np.sin(phi), np.sin(theta))
ze = re * np.outer(np.ones(np.size(phi)), np.cos(theta))

fig = go.Figure()

fig.add_trace(go.Surface(x=xm, y=ym, z=zm, showscale=False))

fig.add_trace(go.Surface(x=xe, y=ye, z=ze, showscale=False))

fig.add_trace(
    go.Scatter3d(x = x, y = y, z = z, mode = 'lines', marker=dict(size=3), name='Orbit')) 

fig.update_layout(
    scene = dict(
        xaxis = dict(nticks=10, range=[-20000, 385000],),
                     yaxis = dict(nticks=10, range=[-20000, 20000],),
                     zaxis = dict(nticks=10, range=[-50000, 80000],),),
    width=700,
    margin=dict(r=20, l=10, b=10, t=10))

fig.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

mu = 0.1
R = 1
sun_pos = np.array([-mu*R, 0])
earth_pos = np.array([(1-mu)*R, 0])

N = 1000
x, y = np.meshgrid(np.linspace(-1.5, 1.5, N), np.linspace(-1.5, 1.5, N))

term1 = -(1-mu) / ((x + mu)**2 + y**2)**0.5
term2 = -mu / ((x - (1-mu))**2 + y**2)**0.5
term3 = -0.5 * (x**2 + y**2)

u = term1 + term2 + term3

plt.figure(figsize=(5, 5))
levels = np.linspace(-1.8, -1.4, 20)
plt.contour(x, y, u, levels=levels)
plt.scatter([sun_pos[0]], [sun_pos[1]], c='y', s=75)
plt.scatter([earth_pos[0]], [earth_pos[1]], c='b', s=25)
plt.scatter([0], [0], c='k', marker='x')
plt.axis('equal')

plt.savefig("plot09.png", dpi=300, format='png')

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
import matplotlib.pyplot as plt

mu = 0.1
R = 1
sun_pos = np.array([-mu*R, 0])
earth_pos = np.array([(1-mu)*R, 0])

N = 10
x, y = np.meshgrid(np.linspace(-1.5, 1.5, N), np.linspace(-1.5, 1.5, N))

term1 = -(1-mu) / ((x + mu)**2 + y**2)**0.5
term2 = -mu / ((x - (1-mu))**2 + y**2)**0.5
term3 = -0.5 * (x**2 + y**2)
u = term1 + term2 + term3

X = [-1.5, -1.25, -1.0, -0.5, 0, 0.5, 1.0, 1.25, 1.5]
Y = [-1.5, -1.25, -1.0, -0.5, 0, 0.5, 1.0, 1.25, 1.5]

fig = go.Figure(data=go.Contour(
    z=pd.DataFrame(u),
    x=X,
    y=Y,
    showscale=False,
    contours=dict(coloring='lines')
))

fig.add_scatter(x = (sun_pos[0],sun_pos[0]), 
                y = (sun_pos[1],sun_pos[1]), name="Sun", mode = 'markers',
                marker=dict(size=10, color='orange', symbol='circle'))

fig.add_scatter(x = (earth_pos[0],earth_pos[0]), 
                y = (earth_pos[1], earth_pos[1]), name="Earth",  mode = 'markers',
                marker=dict(size=10, color='deepskyblue', symbol='circle'))

fig.add_scatter(x = (0,0), y = (0,0), name="Center",  mode = 'markers',
                marker=dict(size=8, color='limegreen', symbol='circle'))

fig.update_traces(line_width=2, selector=dict(type='contour'))

fig.update_layout(
    plot_bgcolor='midnightblue',
    paper_bgcolor='azure',
    autosize=False,
    width=500,
    height=500)
fig.show()

In [ ]:
pd.DataFrame(u)